In [339]:
from lxml import etree, html as lhtml
import requests
from tqdm.notebook import tqdm
from multiprocessing import Pool
import pandas as pd

In [10]:
#для начала скачаем список авторов
with open("authors.txt" , "r") as authors_file:
        authors_ids = authors_file.readlines()

In [16]:
authors_ids = [i.strip() for i in authors_ids]
print(authors_ids[0:5], len(authors_ids))

['15753', '19769', '15844', '19768', '20948'] 35


In [47]:
#подготовим заготовки url
base_authors_url = "https://www.respublica.ru/authors/"
base =  "https://www.respublica.ru"
base_authors_url + authors_ids[0]

'https://www.respublica.ru/authors/15753'

In [99]:
#подготовим список страничек для обкачки
books_list = []

for num_author in range(len(authors_ids)):
    
    #будем ходить по нормерам страничек из поиска, собирая все ссылки,
    #начинаем с первой, по ней же понимаем сколько 
    #всего страничек (по количеству книжек всего / 22)
    
    working_url = base_authors_url + authors_ids[num_author] + "?page="
    
    #делаем запрос
    r = requests.get(working_url + str(1))
    
    #добавляем в список все ссылки на книжки, которые нашлись на страничке
    books_page = r.text
    tree = lhtml.fromstring(books_page)
    books_list_tree = tree.xpath('//a[@class="rd-listing-product-item__image-wrapper"]')
    books_list = books_list + [e.attrib['href'] for e in books_list_tree]    
    
    #посчитаем количество страниц всего
    num_pages = tree.xpath('//span[@class="rd-listing-count__total"]')[0].text
    num_pages = (int(num_pages) - 1)//22 + 1 
    
    
    #повторим то же для остальных, елси они есть 
    if num_pages > 1:
        for i in range(2,num_pages + 1):
            r = requests.get(working_url + str(i))
            books_page = r.text
            tree = lhtml.fromstring(books_page)
            books_list_tree = tree.xpath('//a[@class="rd-listing-product-item__image-wrapper"]')
            books_list = books_list + [e.attrib['href'] for e in books_list_tree]

In [102]:
#убрали одну из книжек - она встретилась два раза 
#возможно ее писали два автора из списка вместе 

print(len(books_list))
print(len(set(books_list)))
books_list = set(books_list)

2453
2452


In [333]:
def process_page(url):
    '''
    Достает нужную информацию из странички 
    '''
    
    r = requests.get(url)
    #качаем до победного
    while r.ok == False:
        r = requests.get(url)
    
    #делаем дерево
    books_page = r.text
    tree = lhtml.fromstring(books_page)
    
    #часть объектов есть на каждой странички, без них страничек не бывает 
    book_info = {
    "Категория":";".join([i.text for i in tree.xpath('//a/span[@itemprop="name"]')]),
    "Название": tree.xpath('//h1[@class="rd-page-product__title"]')[0].text ,
    "Автор": tree.xpath('//a[@itemprop="brand"]')[0].text ,
    "ID":  tree.xpath('//span[@itemprop="sku"]')[0].text,
    "Изображение": base + tree.xpath('//img[@class="rd-page-product__img"]')[0].attrib['src'], 
    "Описание": "".join(list(filter(None,[i.text for i in tree.xpath('//div[@class="rd-page-product__desc-body"]/*/*|//div[@class="rd-page-product__desc-body"]/*|//div[@class="rd-page-product__desc-body"]')]))),
    "Цена": int(''.join(tree.xpath('//span[@class="num"]')[0].text.split())),
    }

    
    #другие поля опциональны 
    #книга может быть или не быть в наличии
    buy = tree.xpath('//span[@class="rd-page-product__buy-text"]')[0].text 
    if buy == "Сообщить о поступлении":
        book_info["Наличие"] = False
    else:
        book_info["Наличие"] = True

        
    # Превью есть не у всех книжек
    preview = tree.xpath('//a[@class="download-pdf"]')
    if preview:
        book_info["Превью"] = preview[0].attrib['href']

        
    #На случай скидок
    old_price = tree.xpath('//span[@class="prev"]') 
    if old_price:
        book_info["Цена (старая)"] = int(''.join(old_price[0].text.split()))

        
    #Таблицы с характеричтиками отличаются по набору 
    table_elems = tree.xpath('//p/span[@itemprop="name"]')
    table_values = tree.xpath('//p/*[@itemprop="value"]')

    for i in range(len(table_elems)):
        book_info[table_elems[i].text] = table_values[i].text

    #Отзывы есть далеко не у всех, их считаем по числу звезд/число отзывов    
    num_review = tree.xpath('//div[@class="rd-product-reviews__item-author"]')
    if num_review:
        book_info["Число отзывов"] = len(num_review)
        book_info["Оценка"] = len(tree.xpath('//div[@class="rd-product-reviews__item-stars"]/div[@class="rd-rating-stars"]/div[@class="rd-rating-stars__star rd-rating-stars__star_fill"]')) / len(num_review)

        
    #Элемент, кажется, совсем не обязательный тк нельзя оставить отзыв без комментария на сайте республики
    num_marks = tree.xpath('//div[@class="rd-product-reviews__item-body"]')
    if num_marks:
        book_info["Число оценок"] = len(num_marks)

    return(book_info)
    

In [304]:
#Подготовим url из того списка, что мы нашли 
urls = [base + i for i in books_list]
urls[0:3]

['https://www.respublica.ru/detyam/detskie-knigi/poznavatelnaya-literatura/476878-kumon-nauka',
 'https://www.respublica.ru/knigi/hudozhestvennaya-literatura/sovremennaya-proza/392501-ray-na-zakaz',
 'https://www.respublica.ru/knigi/hudozhestvennaya-literatura/fantastika-i-fentezi/469389-marsianskie-hroniki']

In [334]:
#Сделаем несколько процессов и проведем обкачку 
#С баром конечно не очень красиво получилось, но мы же знаем сколько у нас страничек было

with Pool(processes=10) as pool:
    pages = list(tqdm(pool.imap(process_page, urls)))
pool.join()

In [337]:
#Проверим словари 
print(len(pages))
pages[0:5]

2452


[{'Категория': ' Детям; Детские книги; Познавательная литература',
  'Название': 'Kumon. Наука',
  'Автор': 'Тору Кумон',
  'ID': '476878',
  'Изображение': 'https://www.respublica.ru/uploads/00/00/00/6d/2m/large_b099858c6397defa.jpg',
  'Описание': 'Тетрадь «Kumon. Наука» познакомит вашего ребёнка с основами таких естественных наук, как биология, география, физика и астрономия. Интересные и разноплановые задания помогут расширить кругозор и привить интерес к учёбе, а работа с карандашом и наклейками будет способствовать развитию мелкой моторики рук.Фишки книги:— 47 заданий из разных научных сфер;— Основы четырех естественных наук: биология, география, астрономия, физика;— Шесть типов заданий: приклей наклейку, пройди лабиринт, раскрась, нарисуй, соедини линиями, найди нужную картинку.— Каждое задание — это не только какие-то интересные факты, но и тренировка мелкой моторики рук;— В книге есть страницы с наклейками, которые легко отделяются по перфорации;— «Окошечки», в которые необход

In [377]:
#сделаем датафрейм, отсортируем по индексам
df = pd.DataFrame(pages)
df.sort_values(by=["ID"], inplace=True)
df.index  = df.ID
df.drop('ID', axis = 1, inplace = True)

In [357]:
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', 200)

In [378]:
#все ок 
df.head(200)

,Категория,Название,Автор,Изображение,Описание,Цена,Наличие,ISBN,Издательство,Серия,Жанр,Возраст,Обложка,Формат,Количество страниц,Год издания,Язык,Раздел,Страна-производитель,Превью,Направление,"Вес, г",Возрастные ограничения,Упаковка,Материал,Тип,Иллюстратор,Цена (старая),Число отзывов,Оценка,Число оценок,Рисунок,"Размер, см",Герои,Вид бумаги,Иллюстрации,Пол,Тематика,Назначение,Эпоха
ID,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
000033,Книги; Художественная литература; Поэзия,Много шума из ничего,Уильям Шекспир,https://www.respublica.ru/uploads/00/00/00/cq/...,В настоящем издании под одной обложкой объедин...,100,False,978-5-389-03203-3,Азбука,NaN,NaN,NaN,Мягкая,11 х 18,224,2015,Русский,Зарубежные,Россия,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,140.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
000056,Книги; Художественная литература; Поэзия,Двенадцатая ночь,Уильям Шекспир,https://www.respublica.ru/uploads/01/00/00/24/...,Перу великого английского драматурга Уильяма Ш...,100,False,978-5-389-03202-6,Азбука,NaN,NaN,NaN,Мягкая,11 х 18,192,2011,Русский,Зарубежные,Россия,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,140.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
021678,Книги; Художественная литература; Книги на ин...,N or M?,Agatha Christie,https://www.respublica.ru/uploads/00/00/00/68/...,Tommy & Tuppence are hired to track down warti...,610,False,NaN,HarperCollins,NaN,Детектив,NaN,Мягкая,10 х 17,304,2001,Английский,Зарубежные,США,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
021679,Книги; Художественная литература; Книги на ин...,The Secret Adversary,Agatha Christie,https://www.respublica.ru/uploads/00/00/00/68/...,Agatha Christie's first Tommy and Tuppence mys...,610,False,NaN,HarperCollins,NaN,Детектив,NaN,Мягкая,11 х 17,400,2011,Английский,Зарубежные,США,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
021680,Книги; Художественная литература; Книги на ин...,By the Pricking of My Thumbs,Agatha Christie,https://www.respublica.ru/uploads/01/00/00/68/...,An old woman in a nursing home speaks of a chi...,510,False,NaN,HarperCollins,NaN,Детектив,NaN,Мягкая,11 х 17,352,2011,Английский,Зарубежные,США,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,610.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
021681,Книги; Художественная литература; Книги на ин...,Partners in Crime,Agatha Christie,https://www.respublica.ru/uploads/00/00/00/68/...,Agatha Christie's complete Tommy and Tuppence ...,610,False,NaN,HarperCollins,NaN,Детектив,NaN,Мягкая,11 х 17,352,2001,Английский,Зарубежные,США,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
021700,Книги; Художественная литература; Книги на ин...,Evil Under the Sun,Agatha Christie,https://www.respublica.ru/uploads/00/00/00/0p/...,The beautiful bronzed body of Arlena Stuart la...,540,True,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
021702,Книги; Художественная литература; Книги на ин...,The Murder on the Links,Agatha Christie,https://www.respublica.ru/uploads/01/00/00/68/...,"On a French golf course, a millionaire is foun...",639,False,NaN,HarperCollins,NaN,NaN,NaN,Мягкая,13 x 20,288,2017,NaN,NaN,США,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
021703,Книги; Художественная литература; Книги на ин...,The ABC Murders,Agatha Christie,https://www.respublica.ru/uploads/01/00/00/5d/...,"There's a serial killer on the loose, working ...",610,False,NaN,HarperCollins,NaN,Детектив,NaN,Мягкая,11 х 17,278,2010,Английский,Зарубежные,США,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [380]:
#сохраним результаты 
with open('hw_3.csv', mode='w', encoding='utf-8') as f_csv:
    df.to_csv(f_csv, index=False)